In [2]:
import torch

text is copied from https://www.theguardian.com/football/2024/oct/31/ruud-van-nistelrooy-manchester-united-future

In [3]:
text = "Ruud van Nistelrooy is once again the focal point at Manchester United, almost two decades after ending a five-year spell leading the line. The boots were hung up long ago and he has swapped the famous red shirt for a tunic and long coat but it is like the old days as the Old Trafford fans elongate his first name. Ruuuuuuuud was the chant when he left the tunnel to take charge of his first match, leading United to a much-needed 5-2 Carabao Cup victory over Leicester. United laid down a marker but with Chelsea’s visit on Sunday and a first Europa League win of the season required against Paok four days later, the Dutchman knows this is not a busman’s holiday, even for someone who has maintained his hero status despite an acrimonious exit in 2006. I want to help, I want to do everything I can in my abilities, as many others do, to fight for this club,” said Van Nistelrooy, who returned in July as Erik ten Hag’s assistant. “The club is in a difficult situation and it is not where we all want it to be. The challenge is there. Everyone is looking at Man United and the situation it is in but it is up to us as people on the ground to do the utmost, supported by fans. It is a process and it is something to work for and do everything for. With Rúben Amorim waiting to join from Sporting in November’s international break, the caretaker manager knows his fate and timeframe, alleviating pressure. It is, on the other hand, a chance for Van Nistelrooy to prove his worth as a member of the backroom staff. Ole Gunnar Solskjær might have thought he had little chance of becoming the permanent manager when he stepped in after José Mourinho was dismissed. It helped the Norwegian that he was parachuted in from the outside as United looked to steady the ship with a legendary figure, whereas Van Nistelrooy has been on the touchline all season behind Ten Hag, watching the team fail. Van Nistelrooy has similar connections to Solskjær’s and spoke before his Old Trafford managerial debut to Sir Alex Ferguson, another figurehead who reminds supporters of better days. It was special to come back to the club and the city, Van Nistelrooy said. I enjoy being around Manchester, the players and staff – not many [staff from his playing days are] still around. I decided to come back here for a very important reason: that I am here as an assistant. I came here to help the club move forward and I am still very motivated to do so in any capacity, as an assistant and now an interim manager. After that I go back to my assistant contract that I have for this season and next. Van Nistelrooy followed a similar path to Ten Hag with his formation against Leicester. This is not the time for revolutionary thinking; that should come from Amorim. Pressure had been built up before Ten Hag’s tenure stumbled to its final demise at West Ham and the players looked more relaxed against Leicester, but they know Sunday poses a far more difficult equation. This is no audition for Van Nistelrooy and his popularity will not wane, regardless of the next three games, but he will want to stand aside as a legend, whether he moves back into the shadows or exits Old Trafford to make way for Amorim’s backroom staff. Unlike Lee Carsley during his time in charge of England, he will not have to deal with questions about a desire to take the job because everyone knows what comes next and when. Van Nistelrooy found out during a spell at PSV about the complexities of managing a big club that he knows well. Somehow he will have to balance the enjoyment of having his dream job with the knowledge of its brevity and the need to get results to leave United in a better place."

In [4]:
print(f"length of the text - {len(text)}")

length of the text - 3663


Using pythons Unicode text conversion functions to covert each char of this text to int

In [5]:
text_ints = [ord(c) for c in text]
len(text_ints)

3663

Using built-in  UTF-8 encodings to convert text to embeddings----------

With utf-8 embeddings, we get int for each char in the text. As we are using 8-byte for encoding, max vocab size can be 255.
So model has to deal to with long sequences of int even for short text..

In [6]:
tokens = list(text.encode("utf-8"))
tokens = list(map(int, tokens))
len(tokens)


3687

introducing byte-pair encoding (BPE) to reduce sequence length -------

This will helps us "pack" more information (more text) in smaller number of ints, i.e. we can then use model's context length effectively.

Summary of BPE: find the bytes pairs (here bytes are converted to ints) that occur most often and replace them with a new id (here a new int).
Repeat this process "desired number of times". Every time you replace a byte pair by a new id, your vocab size increases (so the size of model's embedding table) and your input sequence length reduces.

Toy example of BPE: https://en.wikipedia.org/wiki/Byte_pair_encoding

Programatically BPE has 2 main parts:
- finding repeating byte-pairs 
- replacing the most frequently repeating byte-pair

In [7]:
def find_bp_stats(text_list):
  """
  finds repeating byte pair and their frequency
  Args:
      text_list (_type_): list containing int for each char in the text
  """
  counts = {}
  for idx in range(0, len(text_list)-1, 1):
    bp = (text_list[idx], text_list[idx+1])
    counts[bp] = counts.get(bp, 0) +1
  # sort the bp wrt their frequency
  # counts = sorted(((v, k) for k, v in counts.items()), reverse=True)
  return counts

stats = find_bp_stats(tokens)
print(stats)

{(82, 117): 2, (117, 117): 8, (117, 100): 3, (100, 32): 62, (32, 118): 5, (118, 97): 2, (97, 110): 67, (110, 32): 53, (32, 78): 11, (78, 105): 9, (105, 115): 47, (115, 116): 39, (116, 101): 40, (101, 108): 19, (108, 114): 9, (114, 111): 22, (111, 111): 15, (111, 121): 11, (121, 32): 29, (32, 105): 37, (115, 32): 86, (32, 111): 22, (111, 110): 28, (110, 99): 6, (99, 101): 9, (101, 32): 104, (32, 97): 82, (97, 103): 15, (103, 97): 7, (97, 105): 11, (105, 110): 49, (32, 116): 89, (116, 104): 63, (104, 101): 70, (32, 102): 35, (102, 111): 22, (111, 99): 2, (99, 97): 7, (97, 108): 12, (108, 32): 17, (32, 112): 14, (112, 111): 8, (111, 105): 2, (110, 116): 18, (116, 32): 70, (97, 116): 26, (32, 77): 4, (77, 97): 3, (99, 104): 14, (101, 115): 21, (101, 114): 45, (114, 32): 34, (32, 85): 7, (85, 110): 7, (110, 105): 8, (105, 116): 35, (101, 100): 27, (100, 44): 4, (44, 32): 24, (108, 109): 1, (109, 111): 12, (111, 115): 4, (116, 119): 1, (119, 111): 3, (111, 32): 45, (32, 100): 23, (100, 101):

In [8]:
def merge_bp(curr_txt, curr_bp, new_id):
  """
  replaces given bp in curr_txt with new_id
  Args:
      curr_txt (_type_): text in the form ints
      curr_bp (_type_): bp to be replaced by new id
      new_id (_type_): id to be assigned when replacing bp
  """
  new_text = []
  i = 0
  replacement_count = 0
  while i < len(curr_txt):
    if i < len(curr_txt) - 1 and curr_txt[i] == curr_bp[0] and curr_txt[i+1] == curr_bp[1]:
      new_text.append(new_id)
      i += 2
      replacement_count += 1
    else:
      new_text.append(curr_txt[i])
      i += 1
  # print(f"replacement counts - {replacement_count}")
  return new_text
      

We know byte pair (101, 32) repeats 104 times, let's try to replace it.
length of our text is 3687, if we replace (101, 32) -> we should have len(new_text) == 3687-104*2+104 = 3583

In [9]:
print(len(tokens))
t2 = merge_bp(tokens, (101, 32), 3688)
print(len(t2))
3688 in t2


3687
3583


True

Now let's perform BPE on whole text N times. 
To decide N, either we can treat vocab_size as y hyperparameter. As we are using utf-8 encoding, we have 256 unique bytes in ourvocab,
so vocab_size = 256, lets expand vocab size by 25 by doing 25 merges

In [ ]:
vocab_size = 281 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = find_bp_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge_bp(ids, pair, idx)
  merges[pair] = idx

NameError: name 'tokens' is not defined

let's see how much text size reduction we could obtain


In [11]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 3687
ids length: 2688
compression ratio: 1.37X


This merging process is also called as "training" of the tokenizer. In this process we try to strike a sweet-spot between vocab_size and length of text after encoding using the tokenizer. 

Let's assume that we are happy after one round of training and see how can we use this tokenizer for encoding and decoding the text.

In [12]:
# let's build vocab for our encode-decode experiment
# why 256 -- we use utf-8 encoding, 8 bytes so 256 elements
# and why bytes -- utf-8 has mapping of  bytes:chars, so we also need mapping of int:bytes

vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]


In [13]:
def decode(ids):
  """ given ids (list of integers), return Python string

  Args:
      ids: list of ints to be decoded

  Returns:
      text: decoded text i.e. int -> bytes -> chars mapping 
  """
  
  # let's find bytes belonging to each id in ids and join them as a str
  tokens = b"".join(vocab[idx] for idx in ids)
  # lets decode this str of bytes using utf-8 format, remember utf-8 has mapping of bytes->chars
  # also error handling with errors="replace" as not every char will have a valid utf-x encoding/decoding, "replace" then catches
  # the errors and replaces them with "?"
  text = tokens.decode("utf-8", errors="replace")
  return text

In [14]:
def encode(text):
  """given a string, return list of integers (the tokens)

  Args:
      text : list of str to be encoded

  Returns:
      _type_: int id for each char in str
  """
  # convert the text into str of bytes
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    # let's find stats of repetative byte-pairs
    stats = find_bp_stats(tokens)
    # finding most eligible token for merging
    
    # Get the list of pairs that are in both stats and merges
    available_pairs = [p for p in stats if p in merges]
    # now merges is a dict of (int, int) -> priority_num. lower the priority_num -> earlier that (int, int) pairing was added to merges.
    # in short we select pair for merging in the same order as comapred to merges
    pair = min(available_pairs, key=lambda p: merges[p], default=None)
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge_bp(tokens, pair, idx)
  return tokens

print(encode(""))

[]


let's test our encoding and decoding

In [15]:
valtext = "Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes thousands of emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters."
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

True


Using regex to meaningfully split the text before tokenization, refer karpathy's video for [detailed explanation](https://www.youtube.com/watch?v=zduSFxRajkE&t=3456s)

my understanding: Until now we have been spliting words almost without any logic. e.g. the most frequently occurring toeken (101, 32) was "e. ".
To split words meaningfully, a regex statement is used which --
- splits words at spaces, 
- makes sequences of numbers 
- separates numbers from words
- separates punctuations


In [5]:
# for BPE a simpler regex should suffice, e.g.
import regex as re
bpe_pat = re.compile(r"\p{N}+|\p{L}+|[^\p{L}\s]")
print(re.findall(bpe_pat, "Why'd heck we need tokenizer12?345?"))

['Why', "'", 'd', 'heck', 'we', 'need', 'tokenizer', '12', '?', '345', '?']


In [ ]:

gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
# ##### this regex experession is much more like sentencepiece though.. Notice those spaces being included in the words?? 
# That's how sentencepiece pre-processes the input.
print(re.findall(gpt2pat, "Why'd heck we need tokenizer12?345?"))

['Why', "'d", ' heck', ' we', ' need', ' tokenizer', '12', '?', '345', '?']
